## Problema del Transporte

¿Cuál es el esquema óptimo de transporte entre las fábricas y los mercados que minimiza el coste?


### Formulación matemática

* $f \in F$: fábricas
* $m \in M$: mercados
* $c_{fm}$: costes de transporte entre cada fábrica y mercado
* $u_{fm}$: unidades transportadas entre cada fábrica y mercado
* $d_m$: demanda de cada mercado
* $p_f$: tope de producción de cada fábrica

\begin{eqnarray*}
\min_{u_{fm}} & \sum_{f \in F} \sum_{m \in M} c_{fm} u_{fm} \\
\text{s.t.} & \\
  & \sum_{f \in F} u_{fm} \geq d_{m} & \forall m \in M \\
  & \sum_{m \in M} u_{fm} \leq p_{f} & \forall f \in F \\
  & u_{fm} \geq 0 & \forall f \in F, \forall m \in M
\end{eqnarray*}



### Datos

Distancia (100 km) | M1 | M2 | M3 | oferta (uds)
  -|-|-|-|-
  F1  | 2.5 | 1.7 | 1.8 | 350
  F2  | 2.5 | 1.8 | 1.4 | 600
  demanda (uds) | 325 | 300 | 275 |

* Coste por distancia: 90 €/100 km

In [ ]:
from pandas import DataFrame, Series

In [ ]:
# Coste de cada unidad transportada (€/100 km)
transported_unit_cost = 90

# Listas con las fábricas y mercados disponibles
factories = ["F1", "F2"]
markets = ["M1", "M2", "M3"]

print(transported_unit_cost, factories, markets)

90 ['F1', 'F2'] ['M1', 'M2', 'M3']


In [ ]:
# Serie con la capacidad de fabricación de cada fábrica
offers = Series(index=factories, data=[350, 600])
offers

F1    350
F2    600
dtype: int64

In [ ]:
# Serie con la demanda de cada mercado
demands = Series(index=markets, data=[325, 300, 275])
demands

M1    325
M2    300
M3    275
dtype: int64

In [ ]:
# Tabla con la distancia entre cada fábrica y mercado
distances = DataFrame(index=factories, columns=markets, data=[[2.5, 1.7, 1.8], [2.5, 1.8, 1.4]])
distances

,M1,M2,M3
F1,2.5,1.7,1.8
F2,2.5,1.8,1.4


### Resolución con PuLP

In [ ]:
!pip install pulp
from pulp import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 78.2 MB/s eta 0:00:00


In [ ]:
# Crear problema de minimización
prob = LpProblem("transport_problem", LpMaximize)

# Crear variables de unidades transportadas entre cada fábrica y mercado
transported_units = LpVariable.dicts("Units", indices=(factories, markets), lowBound=0, cat=LpInteger)

# Añadir función objetivo al problema
prob += (lpSum([transported_unit_cost * transported_units[f][m] * distances[m][f] for f in factories for m in markets]), "Coste transporte")

# Añadir restricciones de no sobrepasar producción de fábricas
for f in factories:
    prob += (lpSum([transported_units[f][m] for m in markets]) >= offers[f], f"Tope fábrica {f}")

# Añadir restricciones de satisfacer demandas de mercados
for m in markets:
    prob += (lpSum([transported_units[f][m] for f in factories]) >= demands[m], f"Demanda mercado {m}")

# Comprobar que el problema está bien formulado
prob

transport_problem:
MAXIMIZE
225.0*Units_F1_M1 + 153.0*Units_F1_M2 + 162.0*Units_F1_M3 + 225.0*Units_F2_M1 + 162.0*Units_F2_M2 + 125.99999999999999*Units_F2_M3 + 0.0
SUBJECT TO
Tope_fábrica_F1: Units_F1_M1 + Units_F1_M2 + Units_F1_M3 >= 350

Tope_fábrica_F2: Units_F2_M1 + Units_F2_M2 + Units_F2_M3 >= 600

Demanda_mercado_M1: Units_F1_M1 + Units_F2_M1 >= 325

Demanda_mercado_M2: Units_F1_M2 + Units_F2_M2 >= 300

Demanda_mercado_M3: Units_F1_M3 + Units_F2_M3 >= 275

VARIABLES
0 <= Units_F1_M1 Integer
0 <= Units_F1_M2 Integer
0 <= Units_F1_M3 Integer
0 <= Units_F2_M1 Integer
0 <= Units_F2_M2 Integer
0 <= Units_F2_M3 Integer

In [ ]:
# Resolver el problema, pintar la solución y ver cuánto da de objetivo
status_num = prob.solve()
status_str = LpStatus[status_num]
if status_str == "Optimal":
  print("Problema resuelto óptimamente")
  for v in prob.variables():
      print(f"\tVariable {v.name} = {value(v)}")
  print(f"\tFunción objetivo = {round(value(prob.objective), 2)}")
else:
  print(f"Problema no resuelto óptimamente: {status_str}")

Problema no resuelto óptimamente: Unbounded


* ¿Hay alguna fábrica que no se use a pleno rendimiento? ¿Qué pasaría si en lugar de minimizar los costes buscáramos maximizarlos?
* ¿Y si volviésemos a minimizar los costes, pero en las restricciones dijésemos que las demandas se deben satisfacer de forma exacta?
* ¿Y si volviésemos a maximizar los costes, pero además las demandas que nos dan fueran las mínimas que se deben satisfacer, y las producciones de cada fábrica también fuesen las mínimas en lugar de las máximas?